In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmi-model-3-label-encoder/label_encoder.pkl
/kaggle/input/cmi-model-3-utilities/train_universe.csv
/kaggle/input/cmi-model-3-utilities/best_fold3.pt
/kaggle/input/cmi-model-3-utilities/best_fold2.pt
/kaggle/input/cmi-model-3-utilities/best_fold0.pt
/kaggle/input/cmi-model-3-utilities/best_fold1.pt
/kaggle/input/cmi-model-3-utilities/best_fold4.pt


# **Libraries**

In [2]:
import os
import torch
import kagglehub
from pathlib import Path
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from scipy.spatial.transform import Rotation as R
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.notebook import tqdm
from torch.amp import autocast
import pandas as pd
import polars as pl
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler, LabelEncoder
from transformers import BertConfig, BertModel
import time
import joblib

2025-09-06 22:12:45.500167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757196765.744006      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757196765.815645      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
data_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv'

# **Steps to Convert Raw Dataset → Required Format**

In [4]:

def remove_gravity_from_acc(acc_data, rot_data):
    """
    acc_data: DataFrame or ndarray with columns ['acc_x','acc_y','acc_z']
    rot_data: DataFrame or ndarray with columns ['rot_x','rot_y','rot_z','rot_w']
    Returns ndarray (N,3) linear acceleration (gravity removed) in sensor frame.
    """
    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values.astype(float)
    else:
        acc_values = np.asarray(acc_data, dtype=float)

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values.astype(float)
    else:
        quat_values = np.asarray(rot_data, dtype=float)

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0.0, 0.0, 9.81], dtype=float)

    for i in range(num_samples):
        q = quat_values[i]
        if q is None or np.any(np.isnan(q)) or np.allclose(q, 0.0):
            linear_accel[i, :] = acc_values[i, :]
            continue
        try:
            rotation = R.from_quat(q)
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except Exception:
            linear_accel[i, :] = acc_values[i, :]
    return linear_accel


def angular_velocity_from_quat_with_dt(quat_array, times=None, default_fs=200.0):
    """
    quat_array: (N,4) array in order (rot_x,rot_y,rot_z,rot_w)
    times: optional (N,) timestamps in seconds (or ms; function will normalize)
    default_fs: fallback sampling rate in Hz
    Returns angular_vel (N,3) where last row is zero.
    """
    quat_array = np.asarray(quat_array, dtype=float)
    N = quat_array.shape[0]
    ang_vel = np.zeros((N, 3), dtype=float)
    if N < 2:
        return ang_vel

    if times is not None:
        t = np.asarray(times, dtype=float)
        # heuristic: if timestamps look like epoch ms -> convert
        if np.nanmean(t) > 1e6:
            t = t / 1000.0
        dt = np.diff(t)
        # handle non-positive dt
        pos = dt[dt > 0]
        if pos.size == 0:
            median_dt = 1.0 / default_fs
        else:
            median_dt = np.median(pos)
        dt = np.where(dt > 0, dt, median_dt)
    else:
        dt = np.full(N-1, 1.0 / default_fs)

    for i in range(N-1):
        q_t = quat_array[i]
        q_tp1 = quat_array[i+1]
        if (np.any(np.isnan(q_t)) or np.allclose(q_t, 0.0)) or \
           (np.any(np.isnan(q_tp1)) or np.allclose(q_tp1, 0.0)):
            continue
        try:
            r_t = R.from_quat(q_t)
            r_tp1 = R.from_quat(q_tp1)
            delta = r_t.inv() * r_tp1
            rotvec = delta.as_rotvec()
            ang_vel[i, :] = rotvec / dt[i]
        except Exception:
            # keep zeros on failure
            pass
    return ang_vel


def calculate_angular_distance_with_times(quat_array, times=None):
    """
    Compute per-sample angular distance (angle between consecutive orientations).
    Returns array shape (N,) with last element 0.
    """
    quat_array = np.asarray(quat_array, dtype=float)
    N = quat_array.shape[0]
    ang_dist = np.zeros(N, dtype=float)
    if N < 2:
        return ang_dist

    for i in range(N-1):
        q1 = quat_array[i]
        q2 = quat_array[i+1]
        if (np.any(np.isnan(q1)) or np.allclose(q1, 0.0)) or \
           (np.any(np.isnan(q2)) or np.allclose(q2, 0.0)):
            ang_dist[i] = 0.0
            continue
        try:
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)
            relative = r1.inv() * r2
            ang_dist[i] = np.linalg.norm(relative.as_rotvec())
        except Exception:
            ang_dist[i] = 0.0
    return ang_dist


def compute_tof_region_stats_matrix(subdf_mat, modes):
    """
    subdf_mat: numpy array shape (N,64) with NaNs for missing values
    modes: list of ints, e.g. [4] or [2,4,8,16,32]
    Returns dict mapping column_name -> ndarray (N,)
    """
    new_cols = {}
    mat = subdf_mat.astype(float)  # (N,64)
    with np.errstate(invalid='ignore'):
        new_cols["mean"] = np.nanmean(mat, axis=1)
        new_cols["std"]  = np.nanstd(mat, axis=1)
        new_cols["min"]  = np.nanmin(mat, axis=1)
        new_cols["max"]  = np.nanmax(mat, axis=1)

    flat = mat.reshape(-1, 64)
    for mode in modes:
        if mode <= 0:
            continue
        region_size = 64 // mode if mode>0 and 64%mode==0 else max(1, 64 // mode)
        for r in range(mode):
            start = r * region_size
            end = start + region_size if (r < mode-1) else 64
            region_vals = flat[:, start:end]
            with np.errstate(all='ignore'):
                mean_v = np.nanmean(region_vals, axis=1)
                std_v  = np.nanstd(region_vals, axis=1)
                min_v  = np.nanmin(region_vals, axis=1)
                max_v  = np.nanmax(region_vals, axis=1)
            # keys will be filled by caller with tofid prefix
            new_cols[f"r{mode}_{r}_mean"] = mean_v
            new_cols[f"r{mode}_{r}_std"]  = std_v
            new_cols[f"r{mode}_{r}_min"]  = min_v
            new_cols[f"r{mode}_{r}_max"]  = max_v
    return new_cols


def add_all_tof_features_vectorized(df, tof_mode):
    """
    For each tof sensor 1..5, compute:
      tof_{i}_mean/std/min/max
      and for each mode in modes: tof{mode}_{i}_region_{r}_{stat}
    Returns a new DataFrame = pd.concat([df, new_cols_df], axis=1)
    """
    if tof_mode == 0:
        modes = []
    elif tof_mode == -1:
        modes = [2,4,8,16,32]
    else:
        modes = [tof_mode]

    all_new = {}
    N = len(df)
    for tof_id in range(1, 6):
        tof_cols = [f"tof_{tof_id}_v{p}" for p in range(64)]
        # ensure columns exist (create NaN if missing)
        for c in tof_cols:
            if c not in df.columns:
                df[c] = np.nan

        subdf = df[tof_cols].astype(float).replace(-1, np.nan)
        mat = subdf.values  # (N,64)
        new = compute_tof_region_stats_matrix(mat, modes)
        # prefix names and add to all_new
        all_new[f"tof_{tof_id}_mean"] = new["mean"]
        all_new[f"tof_{tof_id}_std"]  = new["std"]
        all_new[f"tof_{tof_id}_min"]  = new["min"]
        all_new[f"tof_{tof_id}_max"]  = new["max"]
        for mode in modes:
            for r in range(mode):
                all_new[f"tof{mode}_{tof_id}_region_{r}_mean"] = new[f"r{mode}_{r}_mean"]
                all_new[f"tof{mode}_{tof_id}_region_{r}_std"]  = new[f"r{mode}_{r}_std"]
                all_new[f"tof{mode}_{tof_id}_region_{r}_min"]  = new[f"r{mode}_{r}_min"]
                all_new[f"tof{mode}_{tof_id}_region_{r}_max"]  = new[f"r{mode}_{r}_max"]

    # concat once
    new_df = pd.concat([df, pd.DataFrame(all_new, index=df.index)], axis=1)
    return new_df

# -------------------------
# Utility: estimate per-sequence timestamps (seconds)
# -------------------------
def get_sequence_times(group_df, time_col='timestamp'):
    """
    Returns times array in seconds if available, else None.
    If timestamps appear to be epoch-ms, convert to seconds.
    """
    if time_col in group_df.columns:
        times = group_df[time_col].astype(float).values
        if np.nanmean(times) > 1e6:
            times = times / 1000.0
        return times
    return None

# -------------------------
# Main pipeline
# -------------------------
def make_universe_csv(raw_csv_path,
                      out_csv_path="universe.csv",
                      tof_mode=16,
                      default_fs=200.0,
                      time_col='timestamp',
                      tof_raw_keep=True):
    """
    raw_csv_path: path to raw csv (one row per timestamp). Must include sequence_id and sensor columns.
    out_csv_path: final CSV file for CMIFeDataset
    tof_mode: same behavior as CMIFeDataset (0, >1, or -1)
    default_fs: fallback sampling rate for angular velocity if no timestamps present
    time_col: name of timestamp column if present (optional)
    tof_raw_keep: if True keep raw tof_*_v* columns (recommended)
    """
    print("Loading raw CSV:", raw_csv_path)
    df_raw = pd.read_csv(raw_csv_path)
    print("Raw shape:", df_raw.shape)

    # Prepare base: ensure sequence_id present
    if 'sequence_id' not in df_raw.columns:
        raise ValueError("Raw CSV must contain 'sequence_id' column.")

    df = df_raw.copy()

    # ensure base columns exist to avoid KeyError downstream
    for col in ['acc_x','acc_y','acc_z','rot_x','rot_y','rot_z','rot_w']:
        if col not in df.columns:
            df[col] = np.nan
    # ensure thermal columns exist: thm_1..thm_5
    for i in range(1,6):
        c = f"thm_{i}"
        if c not in df.columns:
            df[c] = np.nan

    # ----- IMU: acc_mag, rot_angle (vectorized) -----
    with np.errstate(invalid='ignore'):
        df['acc_mag'] = np.sqrt(df['acc_x'].astype(float)**2 + df['acc_y'].astype(float)**2 + df['acc_z'].astype(float)**2)
        df['rot_angle'] = 2 * np.arccos(df['rot_w'].astype(float).clip(-1,1))

    # compute acc_mag_jerk and rot_angle_vel by group (vectorized groupby diff)
    df['acc_mag_jerk'] = df.groupby('sequence_id')['acc_mag'].diff().fillna(0)
    df['rot_angle_vel'] = df.groupby('sequence_id')['rot_angle'].diff().fillna(0)

    # ----- Linear accel (remove gravity) per sequence -----
    N = len(df)
    lin_x = np.full(N, np.nan, dtype=float)
    lin_y = np.full(N, np.nan, dtype=float)
    lin_z = np.full(N, np.nan, dtype=float)

    if all(c in df.columns for c in ['acc_x','acc_y','acc_z','rot_x','rot_y','rot_z','rot_w']):
        print("Computing gravity-removed linear acceleration per sequence...")
        for seq_id, group in tqdm(df.groupby('sequence_id'), desc="linear_acc"):
            idx = group.index
            acc_sub = group[['acc_x','acc_y','acc_z']].astype(float)
            rot_sub = group[['rot_x','rot_y','rot_z','rot_w']].astype(float)
            la = remove_gravity_from_acc(acc_sub, rot_sub)
            lin_x[idx] = la[:,0]
            lin_y[idx] = la[:,1]
            lin_z[idx] = la[:,2]
    else:
        warnings.warn("Rotation or accelerometer columns missing; using raw acc with gravity approx.")
        if 'acc_x' in df.columns:
            lin_x[:] = df['acc_x'].astype(float).fillna(0.0)
        if 'acc_y' in df.columns:
            lin_y[:] = df['acc_y'].astype(float).fillna(0.0)
        if 'acc_z' in df.columns:
            lin_z[:] = df['acc_z'].astype(float).fillna(0.0) - 9.81

    df['linear_acc_x'] = lin_x
    df['linear_acc_y'] = lin_y
    df['linear_acc_z'] = lin_z
    df['linear_acc_mag'] = np.sqrt(np.nan_to_num(df['linear_acc_x'])**2 + np.nan_to_num(df['linear_acc_y'])**2 + np.nan_to_num(df['linear_acc_z'])**2)
    df['linear_acc_mag_jerk'] = df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0)

    # ----- Angular velocity & angular distance by sequence using timestamps if available -----
    N = len(df)
    av_x = np.zeros(N, dtype=float)
    av_y = np.zeros(N, dtype=float)
    av_z = np.zeros(N, dtype=float)
    ang_dist = np.zeros(N, dtype=float)

    if all(c in df.columns for c in ['rot_x','rot_y','rot_z','rot_w']):
        print("Computing angular velocity & angular distance per sequence...")
        for seq_id, group in tqdm(df.groupby('sequence_id'), desc="ang_vel"):
            idx = group.index
            quat_arr = group[['rot_x','rot_y','rot_z','rot_w']].astype(float).values
            times = get_sequence_times(group, time_col=time_col)  # None if missing
            ang_vel = angular_velocity_from_quat_with_dt(quat_arr, times=times, default_fs=default_fs)
            ad = calculate_angular_distance_with_times(quat_arr, times=times)
            av_x[idx] = ang_vel[:,0]
            av_y[idx] = ang_vel[:,1]
            av_z[idx] = ang_vel[:,2]
            ang_dist[idx] = ad
    else:
        warnings.warn("Rotation quaternion columns missing -> angular velocity/distance set to 0.")

    df['angular_vel_x'] = av_x
    df['angular_vel_y'] = av_y
    df['angular_vel_z'] = av_z
    df['angular_distance'] = ang_dist

    # Fill any remaining NaNs in engineered IMU with zeros (ensures columns exist)
    imu_engineered = ['acc_mag','rot_angle','acc_mag_jerk','rot_angle_vel',
                      'linear_acc_x','linear_acc_y','linear_acc_z',
                      'linear_acc_mag','linear_acc_mag_jerk',
                      'angular_vel_x','angular_vel_y','angular_vel_z',
                      'angular_distance']
    for c in imu_engineered:
        if c not in df.columns:
            df[c] = 0.0
        else:
            df[c] = df[c].fillna(0.0)

    # ----- Thermal basic stats (optional) -----
    thm_cols = [f"thm_{i}" for i in range(1,6)]
    present_thm = [c for c in thm_cols if c in df.columns]
    if present_thm:
        df['thm_mean'] = df[present_thm].astype(float).mean(axis=1)
        df['thm_std']  = df[present_thm].astype(float).std(axis=1).fillna(0.0)
    else:
        warnings.warn("Thermal columns thm_1..thm_5 not found.")

    # ----- ToF features (vectorized, concat once) -----
    print("Computing ToF features (this may take a moment for large files)...")
    df = add_all_tof_features_vectorized(df, tof_mode)

    # Optionally drop raw tof columns (not recommended for baseline)
    if not tof_raw_keep:
        for tof_id in range(1,6):
            for p in range(64):
                c = f"tof_{tof_id}_v{p}"
                if c in df.columns:
                    df.drop(columns=[c], inplace=True)

    # Final save
    print("Final dataframe shape:", df.shape)
    df.to_csv(out_csv_path, index=False)
    print("Saved universe CSV to:", out_csv_path)
    return df


In [6]:
if __name__ == "__main__": 
    RAW_CSV = data_path      
    OUT_CSV = "/kaggle/working/train_universe.csv"     
    TOF_MODE = 16                  # set to same value your dataset config uses (e.g. 4,8,16) or -1 for [2,4,8,16,32]
    DEFAULT_FS = 200.0             

    # run
    universe_df = make_universe_csv(RAW_CSV, OUT_CSV, tof_mode=TOF_MODE, default_fs=DEFAULT_FS, time_col='timestamp', tof_raw_keep=True)


Loading raw CSV: /kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv
Raw shape: (574945, 341)
Computing gravity-removed linear acceleration per sequence...


linear_acc:   0%|          | 0/8151 [00:00<?, ?it/s]

Computing angular velocity & angular distance per sequence...


ang_vel:   0%|          | 0/8151 [00:00<?, ?it/s]

Computing ToF features (this may take a moment for large files)...


/tmp/ipykernel_36/270013593.py:119: RuntimeWarning: Mean of empty slice
  new_cols["mean"] = np.nanmean(mat, axis=1)
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_36/270013593.py:121: RuntimeWarning: All-NaN slice encountered
  new_cols["min"]  = np.nanmin(mat, axis=1)
/tmp/ipykernel_36/270013593.py:122: RuntimeWarning: All-NaN slice encountered
  new_cols["max"]  = np.nanmax(mat, axis=1)
/tmp/ipykernel_36/270013593.py:134: RuntimeWarning: Mean of empty slice
  mean_v = np.nanmean(region_vals, axis=1)
/tmp/ipykernel_36/270013593.py:136: RuntimeWarning: All-NaN slice encountered
  min_v  = np.nanmin(region_vals, axis=1)
/tmp/ipykernel_36/270013593.py:137: RuntimeWarning: All-NaN slice encountered
  max_v  = np.nanmax(region_vals, axis=1)
/tmp/ipykernel_36/270013593.py:119: RuntimeWarning: Mean of empty slice
  new_cols["mean"] = np.nan

Final dataframe shape: (574945, 696)
Saved universe CSV to: /kaggle/working/train_universe.csv
